# Screen Time Mental Health PCA

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.decomposition import PCA

pd.set_option('max_row',2000)
pd.set_option('max_column',300)

base_dir = os.path.join(os.getcwd(), os.pardir)

Get the data.

In [3]:
data_path = os.path.join(base_dir, "data/altered/maps-feature-mapping.csv")
data_df = pd.read_csv(data_path, index_col="X")
data_df

,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,read_wend,work_wend,alon_wend,draw_wend,play_wend,tv_wend,out_win_wend,out_sum_wend,tran_wend,talk_phon_week,text_week,talk_mob_week,comp_week,musi_week,read_week,work_week,alon_week,draw_week,play_week,tv_week,out_win_week,out_sum_week,tran_week,pat_pres_10,pat_pres_8,pat_pres,num_home,mat_anx_1,mat_anx_18m,mat_anx_8m,agg_score,emot_cruel,phys_cruel,mat_anx_0m,pat_ses,mat_ses,pat_edu,mat_edu,parity,dep_band_15,dep_band_13,dep_band_10,dep_band_07,anx_band_15,anx_band_13,anx_band_10,anx_band_07,exercise,child_bull,phone_14_wend,phone_14_week,musi_13,tv_bed_9,own_mob,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
X,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,30.0,59.294132,181.602831,107.0,NaN,NaN,1.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0
2,NaN,9.0,26.0,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
3,NaN,3.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,0.0,0.0,3.0,5.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,1.0,0.0
4,0.0,NaN,22.0,49.812426,160.224186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,2.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,0.0
5,0.0,8.0,31.0,62.270030,191.703227,132.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,NaN,NaN,0.0,4.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,5.0,2.0,2.0,1.0,0.0,0.0,0.0,NaN,2.0,1.0,1.0,NaN,3.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13730,NaN,13.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,0.0,0.0
13731,0.0,6.0,29.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,3.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,0.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0,NaN,9.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,1.0,0.0
13732,NaN,21.0,37.0,58.075832,169.235565,108.0,NaN,NaN,1.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,3.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,3.0,3.0,1.0,2.0,1.0,1.0,1.0,4.0,1.0,0.0,0.0,6.0,1.0,0.0,0.0,2.0,2.0,4.0,4.0,0.0,4.0,0.0,NaN,0.0,3.0,1.0,2.0,1.0,4.0,0.0,1.0,2.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,


#### The percentage of NaNs in each column

In [4]:
d1 =data_df.isnull().sum()/len(data_df)*100
d1.sort_values(ascending = False)

comp_noint_bed_16    96.570555
comp_house           94.808504
dep_thoughts         86.857434
comp_int_bed_16      81.287316
tv_bed_16            71.770788
own_mob              68.064657
prim_diag            67.139945
panic_score          67.139945
secd_diag            67.139945
dep_score            67.139945
has_dep_diag         67.139945
out_sum_wend         64.890054
musi_wend            64.824523
work_wend            64.809961
read_wend            64.802679
talk_mob_wend        64.802679
comp_wend            64.766273
text_wend            64.751711
draw_wend            64.744430
out_win_wend         64.693461
play_wend            64.686180
talk_phon_wend       64.686180
alon_wend            64.686180
tv_wend              64.664337
tran_wend            64.606087
exercise             63.805155
child_bull           63.674093
read_week            63.644969
talk_phon_week       63.601281
musi_week            63.572157
text_week            63.564875
out_win_week         63.543032
alon_wee



#### Replace the NaNs with -1

In [5]:
for c in data_df:
    data_df[c] = data_df[c].fillna(-1)
data_df

,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,read_wend,work_wend,alon_wend,draw_wend,play_wend,tv_wend,out_win_wend,out_sum_wend,tran_wend,talk_phon_week,text_week,talk_mob_week,comp_week,musi_week,read_week,work_week,alon_week,draw_week,play_week,tv_week,out_win_week,out_sum_week,tran_week,pat_pres_10,pat_pres_8,pat_pres,num_home,mat_anx_1,mat_anx_18m,mat_anx_8m,agg_score,emot_cruel,phys_cruel,mat_anx_0m,pat_ses,mat_ses,pat_edu,mat_edu,parity,dep_band_15,dep_band_13,dep_band_10,dep_band_07,anx_band_15,anx_band_13,anx_band_10,anx_band_07,exercise,child_bull,phone_14_wend,phone_14_week,musi_13,tv_bed_9,own_mob,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
X,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-1.0,3.0,30.0,59.294132,181.602831,107.0,-1.0,-1.0,1.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,4.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0
2,-1.0,9.0,26.0,-1.000000,-1.000000,85.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,9.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0
3,-1.0,3.0,24.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,0.0,0.0,0.0,3.0,5.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,0.0,1.0,0.0
4,0.0,-1.0,22.0,49.812426,160.224186,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,1.0,1.0,2.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,0.0
5,0.0,8.0,31.0,62.270030,191.703227,132.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,-1.0,-1.0,0.0,4.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,5.0,2.0,2.0,1.0,0.0,0.0,0.0,-1.0,2.0,1.0,1.0,-1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13730,-1.0,13.0,27.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,9.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,2.0,2.0,0.0,0.0
13731,0.0,6.0,29.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,3.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,0.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0,-1.0,9.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,2.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,2.0,

In [6]:
train_data=np.array(data_df)
train_data

array([[-1.,  3., 30., ...,  2.,  0.,  0.],
       [-1.,  9., 26., ..., -1.,  0.,  1.],
       [-1.,  3., 24., ...,  0.,  1.,  0.],
       ...,
       [-1., 21., 37., ...,  0.,  1.,  0.],
       [ 0., 10., 30., ...,  2.,  1.,  0.],
       [ 0.,  6., 26., ...,  2.,  1.,  0.]])

---
#### Use PCA to reduce dimensionality

In [7]:
def meanX(dataX):
    return np.mean(dataX,axis=0) 


def pca_method(XMat, k):
    average = meanX(XMat)
    m, n = np.shape(XMat)
    data_adjust = []
    avgs = np.tile(average, (m, 1))
    data_adjust = XMat - avgs
    covX = np.cov(data_adjust.T)   
    featValue, featVec=  np.linalg.eig(covX)  
    index = np.argsort(-featValue) 
    finalData = []
    if k > n:
        print ("k must lower than feature number")
        return
    else:
        selectVec = np.matrix(featVec.T[index[:k]]) 
        finalData = data_adjust * selectVec.T
    return finalData

In [8]:
pca_mat=pca_method(train_data,50)
print(pca_mat)
print()
print(np.shape(pca_mat))

[[ 1.32488222e+02 -1.65049226e+00  2.70826596e+00 ...  1.15624310e+00
   9.38102001e-01  3.76452761e-01]
 [-5.38463893e+01 -5.52835009e+01 -2.26310824e+00 ... -1.39549047e-02
   4.82751697e-03  4.54415922e-02]
 [-8.70458408e+01  2.39445216e+01 -1.49053554e+00 ... -1.20810538e-01
  -3.98499618e-04  4.00142610e-02]
 ...
 [ 1.21938100e+02 -7.31956984e+00  1.15074058e+01 ... -2.88245361e-02
  -4.84435219e-01 -5.08723907e-01]
 [-4.61896918e+01 -7.23652583e+01  4.88317047e+00 ...  1.26667091e+00
   1.26891251e+00  9.89611461e-01]
 [ 1.29606799e+02  3.10962844e+00 -3.08144382e+00 ...  1.33699514e-01
  -3.71629397e-01  1.48950543e-01]]

(13734, 50)



#### sklearn inbuilt PCA function

In [9]:
dataMat = np.array(data_df) 
pca = PCA(n_components=50) # or '=mle'
new_data=pca.fit_transform(dataMat)
print(np.shape(new_data))
print()
print(new_data,end='\n')
print()
print(pca.explained_variance_ratio_)

(13734, 50)

[[ 1.32488222e+02  1.65049226e+00 -2.70826596e+00 ...  1.17120359e+00
  -9.01360090e-01  3.80822392e-01]
 [-5.38463893e+01  5.52835009e+01  2.26310824e+00 ... -5.97737536e-03
  -4.72758618e-03  4.16822294e-02]
 [-8.70458408e+01 -2.39445216e+01  1.49053554e+00 ... -9.68287867e-02
  -1.07199614e-03  3.01928232e-02]
 ...
 [ 1.21938100e+02  7.31956984e+00 -1.15074058e+01 ... -9.37763587e-02
   4.39791798e-01 -5.34647659e-01]
 [-4.61896918e+01  7.23652583e+01 -4.88317047e+00 ...  1.35962245e+00
  -1.21404829e+00  9.36938338e-01]
 [ 1.29606799e+02 -3.10962844e+00  3.08144382e+00 ...  9.36861046e-02
   3.81095976e-01  1.36916240e-01]]

[8.15216348e-01 1.65179739e-01 5.01067544e-03 3.93058951e-03
 2.68109493e-03 2.15958394e-03 1.49174324e-03 5.74003236e-04
 5.49277720e-04 4.77109808e-04 2.81807285e-04 2.49047742e-04
 1.90528940e-04 1.65715000e-04 1.41904179e-04 1.10208340e-04
 1.02465216e-04 8.99704910e-05 8.71535078e-05 7.35728686e-05
 6.38674164e-05 6.22392947e-05 5.74611844e-05